In [3]:
# Preprocessing the extrapolated COMPAS dataset

import pandas as pd
from hypothesis_test import *
from kardis_test import *
from mg_calculator import *
from q_finder import *
from data_bin_slicer_multival import *

def excel_to_list(filepath,columns):
    data = pd.read_excel(filepath) 
    df = pd.DataFrame(data,columns=columns)
    array = df.to_numpy()
    data_list = array.tolist()
    return data_list

filepath = r"Datasets/compass_extrapolated_100k.xlsx"
columns= ['sex','race']
observation = excel_to_list(filepath,columns) 
if len(observation) == 100000 and type(observation) == list:
    print('Data processed successfully.')
else:
    print('Data processing failed.')

Data processed successfully.


In [1]:
import csv
import random

file = open(r"Datasets/compas.csv", mode='r', encoding='utf-8-sig')
csvreader = csv.reader(file)
rowlist = [row for row in csvreader]
for category in rowlist[0]:
    index = rowlist[0].index(category)
    exec(category + " = [row[index] for row in rowlist[1:]]")
file.close()

ids = [int(person_id[x]) for x in range(len(person_id))]
observation_100k = [] 

for x in range(100000):
    idx = random.randrange(len(rowlist))
    observation_100k.append(rowlist[idx])

file = open(r"Datasets/people.csv", mode='r', encoding='utf-8-sig')
csvreader = csv.reader(file)
rowlist = [row for row in csvreader]
for category in rowlist[0]:
    index = rowlist[0].index(category)
    exec(category + " = [row[index] for row in rowlist[1:]]")
file.close()

races = [race[x-1] for x in ids]
sex = [sex[x-1] for x in ids]

#print(len(observation_100k))
compas_extrapolated_100k = list(filter(lambda x: not x[1] == 'N/A', list(zip(observax for x in observation_100k,races,sex))))
#print(compas_extrapolated_100k[0])

SyntaxError: Generator expression must be parenthesized (<ipython-input-1-821d0c069aed>, line 31)

In [4]:
def data_bin_slicer_multival(nbdata, nbhypothesis, nbvaluelist, selectedvaluelist):
    """
    This function takes in a list of data with nbvaluelist possible values
    and combines values to reduce it down to len(selectedvaluelist) values. 
    It is used to control the number of possible values for our runtime experiments.
    
    ex. nbvaluelist = [1,2,3,4]
    selectedvaluelist = [1,2]
    nvaluelist = [[1],[2],["not-([1,2])"]]
    """
    nvaluelist = []
    nhypothesis = []
    ndata = []
    biasval_name = '('+ str(selectedvaluelist[0])
    for val in selectedvaluelist[1:]:
        biasval_name += ", " + str(val)
    biasval_name+=')'
    
    for val in range(len(selectedvaluelist)):
        nvaluelist.append(selectedvaluelist[val])
    
    nvaluelist.append(['not-'+biasval_name])
    num_biasval = [nbdata.count(x) for x in selectedvaluelist]
    ncounts = num_biasval
    
    ncounts.append(int(len(nbdata))-int(sum(num_biasval)))
    for x in range(len(selectedvaluelist)):
        ndata += [selectedvaluelist[x]]*num_biasval[x]
    
    ndata += [['not-'+biasval_name]]*ncounts[-1]
            
    for y in range(len(selectedvaluelist)):
        nhypothesis.append(nbhypothesis[nbvaluelist.index(selectedvaluelist[y])])
        
    sum_nhyp = 0
    
    for s in range(len(nhypothesis)):
        sum_nhyp += nhypothesis[s]
    nhypothesis.append(1-sum_nhyp)
    return (ndata,ncounts,nhypothesis,nvaluelist)

In [5]:
# Setup and Experiment

value_list = [['Male','Caucasian'],['Female','Caucasian'],['Male','African-American'],['Female','African-American'],
              ['Male','Hispanic'],['Female','Hispanic'],['Male','Asian'],['Female','Asian'],
              ['Male','Other'],['Female','Other']]
hypothesis = 10*[1/10] #uniform hypothesis
alpha = 0.05 
data = observation
num_values = 1

while num_values < 10:
    selectedvalue = []
    for idx in range(num_values):
        selectedvalue += [value_list[idx]]
    ndata, ncounts, nhypothesis, nvaluelist = data_bin_slicer_multival(data, hypothesis, value_list, selectedvalue)
    my_time = %timeit -n 1 -r 10 -o hypothesis_test_silent(ndata, nvaluelist, alpha, nhypothesis)
    num_values += 1

KeyboardInterrupt: 